# Lab 09: Redes Neuronales Convolucionales (CNN) - Práctica

## Objetivos
1. Implementar convolución 2D desde cero
2. Entender filtros y feature maps
3. Implementar pooling
4. Construir una CNN completa
5. Entrenar CNN en MNIST
6. Visualizar activaciones

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from codigo.cnn import *

np.random.seed(42)

## Parte 1: Convolución 2D desde Cero

### 1.1 Imagen de Ejemplo

In [ ]:
# Crear una imagen simple
imagen = np.array([
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 255, 255, 255, 0, 0],
    [0, 0, 255, 255, 255, 0, 0],
    [0, 0, 255, 255, 255, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0]
], dtype=np.float32)

plt.figure(figsize=(4, 4))
plt.imshow(imagen, cmap='gray')
plt.title('Imagen Original')
plt.colorbar()
plt.show()

### 1.2 Aplicar Filtros Clásicos

In [ ]:
# Obtener filtros clásicos
filtros = obtener_filtros_clasicos()

# Aplicar cada filtro
fig, axes = plt.subplots(2, 3, figsize=(12, 8))
axes = axes.ravel()

for idx, (nombre, kernel) in enumerate(filtros.items()):
    resultado = convolve2d(imagen, kernel)
    
    axes[idx].imshow(resultado, cmap='gray')
    axes[idx].set_title(f'{nombre}')
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

### 🎯 Ejercicio 1.1: Filtros Personalizados

Crea tu propio filtro y aplícalo a la imagen.

In [ ]:
# Tu código aquí
mi_filtro = np.array([
    [0, -1, 0],
    [-1, 5, -1],
    [0, -1, 0]
])

resultado = convolve2d(imagen, mi_filtro)

plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.imshow(imagen, cmap='gray')
plt.title('Original')
plt.subplot(1, 2, 2)
plt.imshow(resultado, cmap='gray')
plt.title('Con Mi Filtro')
plt.show()

## Parte 2: Capa Convolucional

### 2.1 Crear Capa Convolucional

In [ ]:
# Crear capa convolucional
capa_conv = CapaConvolucional(
    num_filtros=8,
    tamano_kernel=3,
    canales_entrada=1,
    stride=1,
    padding=1
)

print(f"Parámetros de la capa:")
print(f"  Filtros: {capa_conv.kernels.shape}")
print(f"  Bias: {capa_conv.bias.shape}")
print(f"  Total parámetros: {capa_conv.kernels.size + capa_conv.bias.size}")

### 2.2 Forward Pass

In [ ]:
# Preparar imagen para la capa (agregar dimensiones batch y canal)
imagen_batch = imagen.reshape(1, 1, 7, 7)

# Forward pass
salida = capa_conv.forward(imagen_batch)

print(f"Forma de salida: {salida.shape}")
print(f"  Batch: {salida.shape[0]}")
print(f"  Canales: {salida.shape[1]}")
print(f"  Altura: {salida.shape[2]}")
print(f"  Ancho: {salida.shape[3]}")

# Visualizar los 8 feature maps
fig, axes = plt.subplots(2, 4, figsize=(12, 6))
axes = axes.ravel()

for i in range(8):
    axes[i].imshow(salida[0, i], cmap='viridis')
    axes[i].set_title(f'Feature Map {i+1}')
    axes[i].axis('off')

plt.tight_layout()
plt.show()

## Parte 3: Pooling

### 3.1 Max Pooling

In [ ]:
# Crear capa de max pooling
max_pool = CapaPooling(tamano_pool=2, stride=2, tipo='max')

# Aplicar pooling a la salida de la convolución
salida_pooled = max_pool.forward(salida)

print(f"Antes de pooling: {salida.shape}")
print(f"Después de pooling: {salida_pooled.shape}")

# Visualizar efecto del pooling
fig, axes = plt.subplots(2, 2, figsize=(10, 10))

axes[0, 0].imshow(salida[0, 0], cmap='viridis')
axes[0, 0].set_title('Feature Map Original')
axes[0, 1].imshow(salida_pooled[0, 0], cmap='viridis')
axes[0, 1].set_title('Después de Max Pooling')

axes[1, 0].imshow(salida[0, 1], cmap='viridis')
axes[1, 0].set_title('Otro Feature Map Original')
axes[1, 1].imshow(salida_pooled[0, 1], cmap='viridis')
axes[1, 1].set_title('Después de Max Pooling')

plt.tight_layout()
plt.show()

### 3.2 Average Pooling

In [ ]:
# Crear capa de average pooling
avg_pool = CapaPooling(tamano_pool=2, stride=2, tipo='average')

# Aplicar pooling
salida_avg_pooled = avg_pool.forward(salida)

# Comparar max vs average pooling
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(salida[0, 0], cmap='viridis')
axes[0].set_title('Original Feature Map')
axes[1].imshow(salida_pooled[0, 0], cmap='viridis')
axes[1].set_title('Max Pooling')
axes[2].imshow(salida_avg_pooled[0, 0], cmap='viridis')
axes[2].set_title('Average Pooling')

plt.tight_layout()
plt.show()

### 🎯 Ejercicio 3.1: Comparar Pooling

¿Cuál es la diferencia principal entre max y average pooling? ¿Cuándo usarías cada uno?

## Parte 4: CNN Completa desde Cero

### 4.1 Crear Arquitectura

In [ ]:
# Crear CNN simple
cnn = CNN()

print("Arquitectura de la CNN:")
print(cnn)

### 4.2 Forward Pass Completo

In [ ]:
# Crear batch de imágenes de ejemplo (28x28 como MNIST)
batch_size = 4
imagenes_ejemplo = np.random.randn(batch_size, 1, 28, 28).astype(np.float32)

# Forward pass
predicciones = cnn.forward(imagenes_ejemplo)

print(f"Shape de entrada: {imagenes_ejemplo.shape}")
print(f"Shape de salida: {predicciones.shape}")
print(f"\nPredicciones (probabilidades para 10 clases):")
print(predicciones)

# Verificar que suman 1 (softmax)
print(f"\nSuma de probabilidades: {predicciones.sum(axis=1)}")

## Parte 5: CNN con PyTorch (Opcional)

### 5.1 Entrenar en MNIST

In [ ]:
# Importar PyTorch (si está disponible)
try:
    import torch
    import torch.nn as nn
    from codigo.cnn import CNNPyTorch, entrenar_cnn_pytorch
    PYTORCH_DISPONIBLE = True
except ImportError:
    print("PyTorch no disponible. Instalar con: pip install torch torchvision")
    PYTORCH_DISPONIBLE = False

In [ ]:
if PYTORCH_DISPONIBLE:
    # Entrenar CNN en MNIST
    print("Entrenando CNN en MNIST...")
    print("Esto puede tomar varios minutos.\n")
    
    modelo, historial = entrenar_cnn_pytorch(epochs=3, batch_size=64)
    
    # Visualizar curvas de entrenamiento
    from codigo.cnn import visualizar_entrenamiento
    visualizar_entrenamiento(historial)
else:
    print("Saltando entrenamiento - PyTorch no disponible")

### 5.2 Visualizar Predicciones

In [ ]:
if PYTORCH_DISPONIBLE:
    from codigo.cnn import visualizar_predicciones
    from torchvision import datasets, transforms
    
    # Cargar datos de test
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])
    
    test_dataset = datasets.MNIST('data', train=False, download=True, transform=transform)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16)
    
    # Obtener un batch
    imagenes, etiquetas = next(iter(test_loader))
    
    # Visualizar predicciones
    visualizar_predicciones(modelo, imagenes, etiquetas)
else:
    print("Saltando visualización - PyTorch no disponible")

### 5.3 Visualizar Filtros Aprendidos

In [ ]:
if PYTORCH_DISPONIBLE:
    from codigo.cnn import visualizar_filtros
    
    # Visualizar filtros de la primera capa
    visualizar_filtros(modelo, capa=0)
else:
    print("Saltando visualización - PyTorch no disponible")

### 🎯 Ejercicio 5.1: Experimentar con Arquitectura

Modifica la arquitectura de CNNPyTorch:
1. Agrega más capas convolucionales
2. Cambia el número de filtros
3. Prueba diferentes tamaños de kernel
4. Compara el rendimiento

In [ ]:
# Tu código aquí


## Parte 6: Cálculo de Dimensiones

### 6.1 Fórmula de Tamaño de Salida

In [ ]:
def calcular_tamano_salida(input_size, kernel_size, stride, padding):
    """
    Calcula el tamaño de salida de una capa convolucional.
    
    Fórmula: output_size = (input_size - kernel_size + 2*padding) / stride + 1
    """
    output_size = (input_size - kernel_size + 2*padding) // stride + 1
    return output_size

# Ejemplos
print("Ejemplos de cálculo de dimensiones:\n")

# Caso 1: Sin padding, stride 1
out = calcular_tamano_salida(input_size=28, kernel_size=5, stride=1, padding=0)
print(f"Input: 28x28, Kernel: 5x5, Stride: 1, Padding: 0")
print(f"Output: {out}x{out}\n")

# Caso 2: Con padding='same'
padding_same = (5 - 1) // 2
out = calcular_tamano_salida(input_size=28, kernel_size=5, stride=1, padding=padding_same)
print(f"Input: 28x28, Kernel: 5x5, Stride: 1, Padding: {padding_same} (same)")
print(f"Output: {out}x{out}\n")

# Caso 3: Stride 2 (reduce tamaño)
out = calcular_tamano_salida(input_size=28, kernel_size=3, stride=2, padding=1)
print(f"Input: 28x28, Kernel: 3x3, Stride: 2, Padding: 1")
print(f"Output: {out}x{out}")

### 🎯 Ejercicio 6.1: Diseñar Arquitectura

Diseña una arquitectura CNN que:
- Empiece con input 32x32x3 (CIFAR-10)
- Termine con output de 10 clases
- Use al menos 3 capas convolucionales
- Calcula las dimensiones en cada capa

In [ ]:
# Tu diseño aquí
print("Arquitectura para CIFAR-10:\n")

# Ejemplo:
# Input: 32x32x3
# Conv1: ?
# Pool1: ?
# Conv2: ?
# ...


## Parte 7: Data Augmentation

### 7.1 Transformaciones Comunes

In [ ]:
if PYTORCH_DISPONIBLE:
    from torchvision import transforms
    from PIL import Image
    
    # Cargar una imagen de MNIST
    test_dataset = datasets.MNIST('data', train=False, download=True)
    img, label = test_dataset[0]
    
    # Definir transformaciones
    transformaciones = {
        'Original': transforms.ToTensor(),
        'Rotación': transforms.Compose([
            transforms.RandomRotation(15),
            transforms.ToTensor()
        ]),
        'Crop': transforms.Compose([
            transforms.RandomResizedCrop(28, scale=(0.8, 1.0)),
            transforms.ToTensor()
        ]),
        'Perspectiva': transforms.Compose([
            transforms.RandomPerspective(distortion_scale=0.2),
            transforms.ToTensor()
        ])
    }
    
    # Aplicar y visualizar
    fig, axes = plt.subplots(1, 4, figsize=(12, 3))
    
    for idx, (nombre, transform) in enumerate(transformaciones.items()):
        img_transformada = transform(img)
        axes[idx].imshow(img_transformada.squeeze(), cmap='gray')
        axes[idx].set_title(nombre)
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()
else:
    print("PyTorch no disponible para data augmentation")

## Resumen y Conclusiones

### Lo que aprendimos:

1. ✅ **Convolución 2D**: Operación fundamental para detectar características
2. ✅ **Filtros**: Kernels para detectar bordes, texturas, patrones
3. ✅ **Pooling**: Reducir dimensionalidad espacial (max, average)
4. ✅ **Arquitectura CNN**: CONV → POOL → CONV → POOL → FC
5. ✅ **Dimensiones**: Cómo calcular tamaños de salida
6. ✅ **Entrenamiento**: CNN en MNIST con PyTorch
7. ✅ **Visualización**: Filtros, activaciones, predicciones

### Conceptos clave:

- **Compartición de parámetros**: Mismo filtro en toda la imagen
- **Invariancia a traslación**: Detecta características en cualquier posición
- **Jerarquía de características**: De bordes a objetos
- **Menos parámetros**: Que redes densas equivalentes

### Próximos pasos:

- Experimentar con arquitecturas más profundas
- Probar en datasets más complejos (CIFAR-10, ImageNet)
- Explorar arquitecturas famosas (ResNet, VGG)
- Transfer learning con modelos pre-entrenados

---

**¡Felicidades! Has completado el laboratorio de CNNs.** 🎉

Continúa con: **[Lab 10: Redes Neuronales Recurrentes y LSTM](../Lab10_Redes_Neuronales_Recurrentes_LSTM/)**